In [ ]:
# Install microdf
!pip install git+https://github.com/PSLmodels/microdf.git

  Cloning https://github.com/PSLmodels/microdf.git to /tmp/pip-req-build-j6ksxh52
  Running command git clone -q https://github.com/PSLmodels/microdf.git /tmp/pip-req-build-j6ksxh52
  Created wheel for microdf: filename=microdf-0.2.0-cp36-none-any.whl size=20051 sha256=11e9ba1059ae7c52760f36230198646b94c48006cb2fcf338d430c0a2b4e0ca4
  Stored in directory: /tmp/pip-ephem-wheel-cache-7oknm_69/wheels/3d/53/af/92e56f83db191b0579d21e8385d61a92a502e66443b23c7e16
Successfully built microdf


In [ ]:
import pandas as pd
import numpy as np
import microdf as mdf

In [ ]:
person = pd.read_csv('https://github.com/UBICenter/ubi-us-states/raw/main/ubi-states.csv.gz')

In [ ]:
# Lower column headers
person.columns = person.columns.str.lower()

In [ ]:
person.columns

Index(['year', 'statefip', 'pernum', 'cpsidp', 'asecwt', 'age', 'race',
       'hispan', 'adjginc', 'taxinc', 'spmtotres', 'spmthresh', 'spmfamunit'],
      dtype='object')

In [ ]:
person.adjginc.replace({99999999: 0},inplace=True)
person.asecwt /= 3

In [ ]:
person['age_group'] = np.where(person.age > 17, 'adult', 'child')

In [ ]:
spmu = person.groupby(['spmfamunit', 'year'])[['adjginc']].sum()
spmu.columns = ['spmu_agi']
person = person.merge(spmu, left_on=['spmfamunit', 'year'], right_index=True)

In [ ]:
person['weighted_agi'] = person.asecwt * person.adjginc

In [ ]:
state = person.groupby(['statefip'])[['weighted_agi', 'asecwt']].sum()
state.columns = ['state_total_agi', 'state_population']
person = person.merge(state, left_on=['statefip'], right_index=True)

In [ ]:
person['state_per_dollar_tax_rate'] = (person.state_population / 
                                       person.state_total_agi)

In [ ]:
population = person.asecwt.sum()
total_agi = (person.adjginc * person.asecwt).sum()
fed_tax_per_dollar = population / total_agi

In [ ]:
# Function input is monthly_ubi, state
# Potential dmemos - race (hispanic), age
# Output poverty rate, change in poverty rate, poverty gap, change in poverty gap,
# gini, change in gini, percent winners, percent losers

In [ ]:
person.columns

Index(['year', 'statefip', 'pernum', 'cpsidp', 'asecwt', 'age', 'race',
       'hispan', 'adjginc', 'taxinc', 'spmtotres', 'spmthresh', 'spmfamunit',
       'age_group', 'spmu_agi', 'weighted_agi', 'state_total_agi',
       'state_population', 'state_per_dollar_tax_rate'],
      dtype='object')

In [ ]:
def pov(df, ubi):
    """ Calculate poverty rate across a set of person records and UBI amount.

    Args:
        df: DataFrame with records for each person.
        ubi: Annual UBI amount.

    Return:
        SPM poverty rate.
    """
    # Calculate required tax rate.
    tax_rate = fed_tax_per_dollar * ubi
    # Add UBI, subtract new tax liability.
    new_spmtotres = df.spmtotres + (ubi * df.pernum) - (df.spmu_agi * tax_rate)
    # Recalculate SPM poverty flags.
    new_pov = new_spmtotres < df.spmthresh
    # Return weighted average of the SPM flags (poverty rate).
    return (new_pov * df.asecwt).sum() / df.asecwt.sum()

In [ ]:
pov(person, 1000)

0.1114224031096988

In [ ]:
pov(person[person.age_group == 'child'], 3600)

0.056291810907975866

In [ ]:
age = person.groupby(['age_group']).apply(lambda x: pov(x, 1000))

age_group  race
adult      100     0.099197
           200     0.173552
           300     0.158102
           651     0.127964
           652     0.136203
           801     0.142902
           802     0.130686
           803     0.080556
           804     0.060389
           805     0.183181
           806     0.068936
           807     0.145310
           808     0.221648
           809     0.096012
           810     0.145610
           811     0.252385
           812     0.069682
           813     0.105784
           814     0.020935
           815     0.000000
           816     0.430923
           817     0.000000
           818     0.000000
           819     0.000000
           820     0.173127
           830     0.000000
child      100     0.098192
           200     0.176796
           300     0.126544
           651     0.096756
           652     0.153165
           801     0.129895
           802     0.080905
           803     0.038091
           804     0.052715
    

In [ ]:
age_state = person.groupby(['age_group', 'statefip']).apply(lambda x: pov(x, 1000))